# Data Exploration

This notebook explores the options chain data in the volsurf database.

**Contents:**
1. Data Quality Checks
2. Options Chain Visualization
3. Liquidity Distribution Analysis

In [ ]:
# Standard imports
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import date, timedelta

# Volsurf imports
from volsurf.database.connection import get_connection
from volsurf.database.schema import init_schema
from volsurf.config.settings import get_settings

# Initialize
init_schema()
conn = get_connection()
settings = get_settings()

print(f"Database: {settings.duckdb_path}")

## 1. Data Overview

In [ ]:
# Check available data
summary = conn.execute("""
    SELECT 
        symbol,
        COUNT(*) as total_records,
        COUNT(DISTINCT quote_date) as trading_days,
        MIN(quote_date) as first_date,
        MAX(quote_date) as last_date,
        SUM(CASE WHEN is_liquid THEN 1 ELSE 0 END) as liquid_records
    FROM raw_options_chains
    GROUP BY symbol
""").fetchdf()

summary

In [ ]:
# Select symbol for analysis
SYMBOL = 'SPY'

# Get latest available date
latest_date = conn.execute(
    "SELECT MAX(quote_date) FROM raw_options_chains WHERE symbol = ?",
    [SYMBOL]
).fetchone()[0]

print(f"Analyzing {SYMBOL} as of {latest_date}")

## 2. Data Quality Analysis

In [ ]:
# Get options chain for latest date
chain_df = conn.execute("""
    SELECT *
    FROM raw_options_chains
    WHERE symbol = ? AND quote_date = ?
    ORDER BY expiration_date, strike
""", [SYMBOL, latest_date]).fetchdf()

print(f"Total options: {len(chain_df):,}")
print(f"Liquid options: {chain_df['is_liquid'].sum():,}")
print(f"Expirations: {chain_df['expiration_date'].nunique()}")
print(f"\nColumns: {list(chain_df.columns)}")

In [ ]:
# Check for data quality issues
quality_checks = {
    'Missing bid': chain_df['bid'].isna().sum(),
    'Missing ask': chain_df['ask'].isna().sum(),
    'Bid > Ask': (chain_df['bid'] > chain_df['ask']).sum(),
    'Zero bid': (chain_df['bid'] == 0).sum(),
    'Negative prices': ((chain_df['bid'] < 0) | (chain_df['ask'] < 0)).sum(),
    'Missing IV': chain_df['implied_volatility'].isna().sum() if 'implied_volatility' in chain_df.columns else 'N/A',
    'Missing OI': chain_df['open_interest'].isna().sum(),
}

pd.DataFrame([quality_checks], index=['Count']).T

In [ ]:
# Bid-ask spread distribution
chain_df['spread'] = chain_df['ask'] - chain_df['bid']
chain_df['spread_pct'] = chain_df['spread'] / chain_df['mid'] * 100

fig = px.histogram(
    chain_df[chain_df['spread_pct'] < 100],  # Filter extreme outliers
    x='spread_pct',
    nbins=50,
    title='Bid-Ask Spread Distribution',
    labels={'spread_pct': 'Spread (% of mid)'}
)
fig.add_vline(x=20, line_dash='dash', line_color='red', annotation_text='20% threshold')
fig.show()

## 3. Options Chain Visualization

In [ ]:
# Get underlying price
underlying_price = chain_df['underlying_price'].iloc[0]
print(f"Underlying price: ${underlying_price:.2f}")

# Select a near-term expiration
expirations = sorted(chain_df['expiration_date'].unique())
near_exp = expirations[min(2, len(expirations)-1)]  # 3rd expiration or closest

print(f"Analyzing expiration: {near_exp}")

In [ ]:
# Filter to selected expiration
exp_df = chain_df[chain_df['expiration_date'] == near_exp].copy()

# Separate calls and puts
calls = exp_df[exp_df['option_type'] == 'CALL'].sort_values('strike')
puts = exp_df[exp_df['option_type'] == 'PUT'].sort_values('strike')

# Plot option prices
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=calls['strike'], y=calls['mid'],
    mode='lines+markers', name='Calls',
    marker=dict(size=5)
))

fig.add_trace(go.Scatter(
    x=puts['strike'], y=puts['mid'],
    mode='lines+markers', name='Puts',
    marker=dict(size=5)
))

fig.add_vline(x=underlying_price, line_dash='dash', line_color='gray',
              annotation_text=f'Spot: ${underlying_price:.2f}')

fig.update_layout(
    title=f'{SYMBOL} Option Prices ({near_exp})',
    xaxis_title='Strike',
    yaxis_title='Mid Price ($)',
    hovermode='x unified'
)

fig.show()

In [ ]:
# Open Interest by strike
fig = go.Figure()

fig.add_trace(go.Bar(
    x=calls['strike'], y=calls['open_interest'],
    name='Call OI', marker_color='green', opacity=0.7
))

fig.add_trace(go.Bar(
    x=puts['strike'], y=puts['open_interest'],
    name='Put OI', marker_color='red', opacity=0.7
))

fig.add_vline(x=underlying_price, line_dash='dash', line_color='gray')

fig.update_layout(
    title=f'{SYMBOL} Open Interest by Strike ({near_exp})',
    xaxis_title='Strike',
    yaxis_title='Open Interest',
    barmode='group'
)

fig.show()

## 4. Liquidity Analysis

In [ ]:
# Liquidity by expiration
liquidity_by_exp = chain_df.groupby('expiration_date').agg({
    'chain_id': 'count',
    'is_liquid': 'sum',
    'open_interest': 'sum',
    'volume': 'sum'
}).reset_index()

liquidity_by_exp.columns = ['Expiration', 'Total', 'Liquid', 'Total OI', 'Total Volume']
liquidity_by_exp['Liquid %'] = (liquidity_by_exp['Liquid'] / liquidity_by_exp['Total'] * 100).round(1)

liquidity_by_exp.head(10)

In [ ]:
# Liquidity heatmap by strike and expiration
liquid_df = chain_df[chain_df['is_liquid']].copy()

# Create pivot for heatmap
heatmap_data = liquid_df.pivot_table(
    index='strike',
    columns='expiration_date',
    values='open_interest',
    aggfunc='sum'
).fillna(0)

# Filter to reasonable strike range
strike_range = (underlying_price * 0.85, underlying_price * 1.15)
heatmap_data = heatmap_data.loc[
    (heatmap_data.index >= strike_range[0]) & (heatmap_data.index <= strike_range[1])
]

fig = px.imshow(
    heatmap_data.T,
    labels=dict(x='Strike', y='Expiration', color='Open Interest'),
    title=f'{SYMBOL} Open Interest Heatmap (Liquid Options)',
    aspect='auto'
)
fig.show()

## 5. Historical Data Coverage

In [ ]:
# Data coverage over time
coverage = conn.execute("""
    SELECT
        quote_date,
        COUNT(*) as total_options,
        SUM(CASE WHEN is_liquid THEN 1 ELSE 0 END) as liquid_options,
        COUNT(DISTINCT expiration_date) as expirations,
        AVG(underlying_price) as underlying
    FROM raw_options_chains
    WHERE symbol = ?
    GROUP BY quote_date
    ORDER BY quote_date
""", [SYMBOL]).fetchdf()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=coverage['quote_date'], y=coverage['total_options'],
    mode='lines', name='Total Options', fill='tozeroy'
))

fig.add_trace(go.Scatter(
    x=coverage['quote_date'], y=coverage['liquid_options'],
    mode='lines', name='Liquid Options', fill='tozeroy'
))

fig.update_layout(
    title=f'{SYMBOL} Data Coverage Over Time',
    xaxis_title='Date',
    yaxis_title='Number of Options',
    hovermode='x unified'
)

fig.show()

## Summary

Key findings from the data exploration:

1. **Data Quality**: [Add observations]
2. **Liquidity**: [Add observations]
3. **Coverage**: [Add observations]